<a href="https://colab.research.google.com/github/EYBishal/Python-Training-Bootcamp/blob/main/16th-Dec-2025/RDD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Partioning&Repartioning").getOrCreate()

In [2]:
data = [
    ("O001","Hyderabad","Electronics",1200,"Delivered"),
    ("O002","Delhi","Clothing",800,"Delivered"),
    ("O003","Mumbai","Electronics",1500,"Cancelled"),
    ("O004","Bangalore","Grocery",400,"Delivered"),
    ("O005","Hyderabad","Grocery",300,"Delivered"),
    ("O006","Delhi","Electronics",2000,"Delivered"),
    ("O007","Mumbai","Clothing",700,"Delivered"),
    ("O008","Bangalore","Electronics",1800,"Delivered"),
    ("O009","Delhi","Grocery",350,"Cancelled"),
    ("O010","Hyderabad","Clothing",900,"Delivered")
]
columns=["order_id","city","category","order_amount","status"]
df = spark.createDataFrame(data,columns)
df.show()

+--------+---------+-----------+------------+---------+
|order_id|     city|   category|order_amount|   status|
+--------+---------+-----------+------------+---------+
|    O001|Hyderabad|Electronics|        1200|Delivered|
|    O002|    Delhi|   Clothing|         800|Delivered|
|    O003|   Mumbai|Electronics|        1500|Cancelled|
|    O004|Bangalore|    Grocery|         400|Delivered|
|    O005|Hyderabad|    Grocery|         300|Delivered|
|    O006|    Delhi|Electronics|        2000|Delivered|
|    O007|   Mumbai|   Clothing|         700|Delivered|
|    O008|Bangalore|Electronics|        1800|Delivered|
|    O009|    Delhi|    Grocery|         350|Cancelled|
|    O010|Hyderabad|   Clothing|         900|Delivered|
+--------+---------+-----------+------------+---------+



In [3]:
df.rdd.getNumPartitions()

2

In [4]:
df_repart=df.repartition(3)
df_repart.rdd.getNumPartitions()

3

In [5]:
df_coalsec=df.coalesce(1)
df_coalsec.rdd.getNumPartitions()

1

In [7]:
filter_df=df.filter(df.city=="Delhi")
select_df=filter_df.select("order_id","order_amount").show()

+--------+------------+
|order_id|order_amount|
+--------+------------+
|    O002|         800|
|    O006|        2000|
|    O009|         350|
+--------+------------+



In [8]:
df_lineage=(
    df.filter(df.status == "Delivered")
    .filter(df.order_amount > 500)
    .select("city","order_amount")
)

In [9]:
df_lineage.count()

6

In [10]:
df_lineage.show()

+---------+------------+
|     city|order_amount|
+---------+------------+
|Hyderabad|        1200|
|    Delhi|         800|
|    Delhi|        2000|
|   Mumbai|         700|
|Bangalore|        1800|
|Hyderabad|         900|
+---------+------------+



**Parsed Logical Plan** – built by the SQL/DataFrame parser from your code.

**Analyzed Logical Plan** – semantic checks and type resolution (columns and data types fixed).

**Optimized Logical Plan** – logical rewrites (pushdown, pruning, simplifications).

**Physical Plan** – concrete execution strategy (how to actually run on the cluster).

In [11]:
df.explain(True)

== Parsed Logical Plan ==
LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Analyzed Logical Plan ==
order_id: string, city: string, category: string, order_amount: bigint, status: string
LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Optimized Logical Plan ==
LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Physical Plan ==
*(1) Scan ExistingRDD[order_id#0,city#1,category#2,order_amount#3L,status#4]

